In [1]:
import findspark
findspark.init('/home/sdmohant/spark-3.0.0-bin-hadoop2.7')

In [2]:
import pyspark
from pyspark import SparkContext,SparkConf

In [3]:
from pyspark.sql import SparkSession

In [4]:
#spark = SparkSession.builder.appName("MyApp").getOrCreate()
#spark = SparkSession.builder.appName("MyName").getOrCreate()
conf = SparkConf().set("spark.cores.max", "16") \
    .set("spark.driver.memory", "16g") \
    .set("spark.executor.memory", "16g") \
    .set("spark.executor.memory_overhead", "16g") \
    .set("spark.driver.maxResultsSize", "0") \
    .set("spark.sql.shuffle.partitions","300")

sc = SparkContext(appName="model6", conf=conf)
spark = SparkSession(sc)

/home/sdmohant/spark-3.0.0-bin-hadoop2.7/python/pyspark/context.py:220: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [5]:
from pyspark.sql.functions import isnan, count, col

In [6]:
df1 = spark.read.options(header = True, inferSchema = True,delimiter=',').csv("AutPapJouCSCI.csv")

In [7]:
df2 = spark.read.options(header = True, inferSchema = True,delimiter=',').csv("Affiliations.csv")

In [8]:
df2.printSchema()

root
 |-- AffiliationId: long (nullable = true)
 |-- AffiliationRank: integer (nullable = true)
 |-- AfNormalizedName: string (nullable = true)
 |-- AfDisplayName: string (nullable = true)
 |-- AfPaperCount: integer (nullable = true)
 |-- AfCitationCount: integer (nullable = true)



In [9]:
drop_cols = ["AfNormalizedName","AfDisplayName"]
df2 = df2.drop(*drop_cols)

In [10]:
#Read PaperAuthorAffiliations Data
PapAuthAffiliations_df = spark.read.options(header = True,delimiter=',',inferSchema=True ).csv("PaperAuthorAffilitions.csv")

In [11]:
PapAuthAffiliations_df.printSchema()

root
 |-- PaperID: long (nullable = true)
 |-- AuthorId: long (nullable = true)
 |-- AffiliationId: long (nullable = true)
 |-- AuthorSequenceNumber: integer (nullable = true)



In [12]:
PapAuthAffiliations_df = PapAuthAffiliations_df.sort("PaperID")

In [14]:
#Joining Paper and Journal Data
df3 = PapAuthAffiliations_df.join(df2, on=['AffiliationId'], how='inner')

In [15]:
#df3.describe().show()

In [16]:
df4 = df3.join(df1, on=['PaperID'], how='inner')

In [18]:
PaperUrls_df = spark.read.options(header = True,delimiter=',',inferSchema=True ).csv("PaperUrl.csv")

In [19]:
PaperUrls_df.show(2)

+----------+----------+
|   PaperID|SourceType|
+----------+----------+
|2514067917|         1|
|2514067917|         1|
+----------+----------+
only showing top 2 rows



In [20]:
df5 = df4.join(PaperUrls_df, on=['PaperID'], how='inner')

In [23]:
PaperRef_df = spark.read.options(header = True,delimiter=',',inferSchema=True ).csv("PaperReferences.csv")

In [24]:
PaperRef_df.show(2)

+----------+-----+----------+-----------+
|        _1|count|   PaperID|PaperRefCnt|
+----------+-----+----------+-----------+
|2252294392|    8|2252294392|          8|
|1479838528|   79|1479838528|         79|
+----------+-----+----------+-----------+
only showing top 2 rows



In [25]:
df6 = PaperRef_df.join(df4, on=['PaperID'], how='inner')

In [27]:
from pyspark.sql.functions import when

In [29]:
drop_cols = ["_1","count"]
df6 = df6.drop(*drop_cols)

In [34]:
drop_cols = ["AffiliationId","AuthorId","AuthorSequenceNumber"]
df3 = df3.drop(*drop_cols)

In [36]:
from pyspark.sql.functions import sum,avg,max,min,mean,count

In [37]:
df3_final = df3.groupBy("PaperID").agg(avg("AffiliationRank").alias("AfRankNew"), \
         avg("AfPaperCount").alias("AfPaperCountNew"), \
         avg("AfCitationCount").alias("AfCitationCntNew"))        
     

In [39]:
df3 = df3.sort("PaperID")

In [40]:
df3.show(30)

+-------+---------------+------------+---------------+
|PaperID|AffiliationRank|AfPaperCount|AfCitationCount|
+-------+---------------+------------+---------------+
|     23|           6580|      344256|       22910762|
|     37|          10827|        7339|          58252|
|     37|          10827|        7339|          58252|
|    108|           8742|        9952|         182434|
|    125|           7338|       25637|         589161|
|    125|           7338|       25637|         589161|
|    125|           7338|       25637|         589161|
|    125|           7338|       25637|         589161|
|    136|           6625|      382321|       28959166|
|    136|           6889|      113027|        6795383|
|    138|           7625|       59139|        1265728|
|    138|           7625|       59139|        1265728|
|    193|           5999|      313662|       19654001|
|    236|           8339|       26270|         959614|
|    266|           6153|      455787|       36817072|
|    285| 

In [41]:
from pyspark.sql.types import *

In [42]:
df3 = df3.withColumn("AffiliationRank",df3["AffiliationRank"].cast(DoubleType()))

In [43]:
df3 = df3.withColumn("AfPaperCount",df3["AfPaperCount"].cast(DoubleType()))

In [44]:
df3 = df3.withColumn("AfCitationCount",df3["AfCitationCount"].cast(DoubleType()))

In [45]:
df3_new = df3.groupBy("PaperID").mean()

In [46]:
df3_new = df3_new.sort("PaperID")

In [49]:
df6 = PaperRef_df.join(df3_new, on=['PaperID'], how='inner')

In [51]:
drop_cols = ["_1","count"]
df6 = df6.drop(*drop_cols)

In [52]:
df8 = df6.join(df1, on=['PaperID'], how='inner')

In [54]:
#df8.columns

In [55]:
df8.show()

+----------+-----------+-------------+--------------------+------------------+--------------------+----------+------------+-------------+--------------+------------+---------------+---------------+------+------------+----------+------+------------+------------------+------------+--------------+-----------------+-----------------+--------------------+--------------------+--------------+
|   PaperID|PaperRefCnt| avg(PaperID)|avg(AffiliationRank)| avg(AfPaperCount)|avg(AfCitationCount)|Paper_Rank|Paper_RefCnt|CitationCount|EstimatedCount|Journal_Rank|Journal_Pap_Cnt|Journal_Cit_Cnt|CSRank|CSPaperCount|CSCitation|CIRank|CIPaperCount|CIPaperCitationCnt|DocTypeIndex|max_AuthorRank|min_AutPaperCount|max_AutPaperCount|min_AutCitationCount|max_AutCitationCount|min_AuthorRank|
+----------+-----------+-------------+--------------------+------------------+--------------------+----------+------------+-------------+--------------+------------+---------------+---------------+------+------------+-----

In [91]:
df8.coalesce(1).write.format('com.databricks.spark.csv').mode("overwrite").save('DataAfterModel-6.csv',header = 'true')